# COMPROBACION DE FIRE MEDIANTE PYTORCH Y MEDIANTE NUMPY CON MATRIX MULTIPLICATIONS
Partial code are copied heavily from https://github.com/pytorch/vision/blob/master/torchvision/models/squeezenet.py  
SqueezeNet Paper:https://arxiv.org/abs/1602.07360  
SqueezeNet 1.1 model from https://github.com/DeepScale/SqueezeNet/tree/master/SqueezeNet_v1.1   
SqueezeNet 1.1 has 2.4x less computation and slightly fewer parameters than SqueezeNet 1.0, without sacrificing accuracy.

Vamos a trabajar ya con fire1 del bloque 1


In [329]:
#some set up
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
import math
import time
from time import sleep, perf_counter as pc
from matplotlib.pyplot import imshow
%matplotlib inline

In [545]:
torch.cuda.is_available()

True

## Aquí tenemos el medidor entre pytorch y numpy adecuado para cada fire

- En esta aproximacion tengo en cuenta el tiempo de definición de capas y la carga de pesos porque se supone que entre fire y fire estas operaciones tiene que hacerse


In [493]:
canales_iniciales=4
canales_contraidos=2
canales_finales=4
acumulado_pytorch=0
acumulado_numpy=0
idea=True
count=100
tamanyo=5
squeeze_activation = nn.ReLU(inplace=True)

for i in range(count):

    imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo))
    weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
    bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)
    weights2a=np.random.randint(10,size=(canales_finales, canales_contraidos,1,1)).astype(np.float32)  
    bias2a=np.random.randint(10,size=(canales_finales,)).astype(np.float32)    
    weights2b=np.random.randint(10,size=(canales_finales, canales_contraidos,3,3)).astype(np.float32)    
    bias2b=np.random.randint(10,size=(canales_finales,)).astype(np.float32)    
    tic=pc()
    squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
    squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
    squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))    

    squeeze2a=nn.Conv2d(canales_contraidos, canales_finales, kernel_size=1, bias=False)
    squeeze2a.weight = nn.Parameter(torch.from_numpy(weights2a))
    squeeze2a.bias = nn.Parameter(torch.from_numpy(bias2a))
    
    squeeze2b=nn.Conv2d(canales_contraidos, canales_finales, kernel_size=3, bias=False, padding=1)
    squeeze2b.weight = nn.Parameter(torch.from_numpy(weights2b))
    squeeze2b.bias = nn.Parameter(torch.from_numpy(bias2b))    

    imagen1  = torch.from_numpy(imagen).float()
    
    salida1=squeeze1(imagen1)
    salida1_activation=squeeze_activation(salida1)
    
    salida2a=squeeze2a(salida1_activation)
    salida2a_activation=squeeze_activation(salida2a)
    salida2b=squeeze2b(salida1_activation)
    salida2b_activation=squeeze_activation(salida2b)    
    salida2_total=torch.cat([salida2a_activation,salida2b_activation], 1)
    salida2_total_a_numpy=salida2_total.detach().numpy()
    toc=pc()
    acumulado_pytorch=toc-tic+acumulado_pytorch
    
    
    tic2=pc()
    n_h=1
    n_w=1
    pesos1=np.transpose(weights1, axes=(1, 0, 2, 3)).reshape(canales_iniciales*n_h*n_w, -1)
    pesos2a=np.transpose(weights2a, axes=(1, 0, 2, 3)).reshape(canales_contraidos*n_h*n_w, -1)

    n_h=3
    n_w=3
    idea2b_pre=weights2b.reshape(-1,canales_contraidos*n_h*n_w)
    pesos2b=np.transpose(idea2b_pre)   
    bt, c_i = imagen.shape[0:2] 
    s=1
    n_h=1
    n_w=1
    k=imagen.shape[2]-n_h+1
    col = np.zeros((bt, c_i , k,k, n_h, n_w))

    for i in range(k):
        i_max = i + s*n_h
        for j in range(k):
            j_max = j + s*n_w
            col[:, :, i, j, :, :] = imagen[:, :, i:i_max:s, j:j_max:s]
    # col = np.transpose(col, axes=(1, 4, 5, 0, 2, 3)).reshape(c_i*n_h*n_w, -1)
    col = col.reshape(c_i*n_h*n_w, -1)
    imagen1_numpy=np.transpose(col)
    salida1_obtenida= np.dot(imagen1_numpy,pesos1)+bias1
    salida1_obtenida_activacion=np.maximum(0,salida1_obtenida)
    imagen1_obtenida = np.transpose(salida1_obtenida_activacion, axes=(1,0)).reshape(canales_contraidos,tamanyo,tamanyo)
    imagen1_obtenida_ampliada=np.pad(imagen1_obtenida,((0,0),(1,1),(1,1)))
    salida2a_obtenida= np.dot(salida1_obtenida,pesos2a)+bias2a
    salida2a_obtenida_activacion=np.maximum(0,salida2a_obtenida)
    c_i = imagen1_obtenida_ampliada.shape[0] 
    n_h=3
    n_w=3
    k=imagen1_obtenida_ampliada.shape[1]-n_h+1
    col = np.zeros(( c_i , k,k, n_h, n_w))

    for i in range(k):
        i_max = i + s*n_h
        for j in range(k):
            j_max = j + s*n_w
            col[ :, i, j, :, :] = imagen1_obtenida_ampliada[ :, i:i_max:s, j:j_max:s]
    col = np.transpose(col, axes=(0, 3, 4, 1, 2)).reshape(c_i*n_h*n_w, -1)
    #col = col.reshape(c_i*n_h*n_w, -1)
    salida_obtenida_2b=np.transpose(col)
    salida2b_obtenida= np.dot(salida_obtenida_2b,pesos2b)+bias2b
    salida2b_obtenida_activacion=np.maximum(0,salida2b_obtenida)
    salida2_total_numpy=np.concatenate((salida2a_obtenida_activacion,salida2b_obtenida_activacion), axis=1)
    imagen_total_obtenida = np.transpose(salida2_total_numpy, axes=(1,0)).reshape(canales_finales*2,tamanyo,tamanyo)
    toc2=pc()
    acumulado_numpy=toc2-tic2+acumulado_numpy
    idea&=np.allclose(salida2_total_a_numpy, imagen_total_obtenida,rtol=1e-01, atol=1e-01)
print(salida2_total_a_numpy)
print(imagen_total_obtenida)
print ("tiempo en segundos con pytorch= ", acumulado_pytorch/count)
print ("tiempo en segundos con numpy= ", acumulado_numpy/count)
print ("pytorch==numpy, total:",idea)


[[[[  655.   762.   865.   917.   581.]
   [  658.   850.   633.  1137.   849.]
   [  686.   998.  1013.   608.  1054.]
   [  882.   753.  1101.   469.  1004.]
   [  845.  1139.   495.   256.   435.]]

  [[  976.  1254.  1360.  1462.   830.]
   [ 1006.  1320.  1042.  1748.  1398.]
   [ 1132.  1524.  1652.   902.  1732.]
   [ 1332.  1076.  1718.   722.  1606.]
   [ 1380.  1856.   762.   396.   646.]]

  [[  874.  1151.  1234.  1331.   735.]
   [  907.  1194.   957.  1576.  1283.]
   [ 1040.  1372.  1512.   807.  1588.]
   [ 1196.   952.  1555.   653.  1463.]
   [ 1264.  1698.   689.   360.   579.]]

  [[  596.   736.   812.   868.   516.]
   [  608.   792.   612.  1052.   820.]
   [  664.   920.   972.   552.  1016.]
   [  808.   668.  1028.   436.   952.]
   [  812.  1092.   460.   240.   396.]]

  [[ 2616.  4840.  5771.  5787.  4839.]
   [ 5178.  8385.  8808.  8336.  5326.]
   [ 5326.  8245.  8463.  9151.  5144.]
   [ 6214.  8829.  7501.  6257.  3909.]
   [ 3194.  4522.  3920.  3125. 

## Veamos ahora fire con opencl

- Vamos a hacer la medición con opencl definido por equipo de squezeenet

#### Step1: OpenCL preparation

In [546]:
# OpenCL setup
import pyopencl as cl
import deviceinfo
from time import time

In [547]:
platform = cl.get_platforms()[2]
print(platform)

device = platform.get_devices()[0]

<pyopencl.Platform 'Intel(R) FPGA Emulation Platform for OpenCL(TM)' at 0x55b6f0890c40>


In [558]:
# Ask the user to select a platform/device on the CLI
context = cl.create_some_context(platform)

# Print out device info
deviceinfo.output_device_info(context.devices[0])

# Create a command queue
queue = cl.CommandQueue(context)

Choose platform:
[0] <pyopencl.Platform 'NVIDIA CUDA' at 0x55b6f0884da0>
[1] <pyopencl.Platform 'Portable Computing Language' at 0x7fdb7ee86008>
[2] <pyopencl.Platform 'Intel(R) FPGA Emulation Platform for OpenCL(TM)' at 0x55b6f0890c40>
[3] <pyopencl.Platform 'Intel(R) FPGA SDK for OpenCL(TM)' at 0x7fd883072060>


Choice [0]: 0


Set the environment variable PYOPENCL_CTX='0' to avoid being asked again.
Device is Tesla V100-PCIE-32GBGPU from NVIDIA Corporation with a max of 80 compute units


In [550]:
tamanyo=55
canales_iniciales=128
canales_contraidos=16
canales_finales=64
imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo))

weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)
weights2a=np.random.randint(10,size=(canales_finales, canales_contraidos,1,1)).astype(np.float32)  
bias2a=np.random.randint(10,size=(canales_finales,)).astype(np.float32)    
weights2b=np.random.randint(10,size=(canales_finales, canales_contraidos,3,3)).astype(np.float32)    
bias2b=np.random.randint(10,size=(canales_finales,)).astype(np.float32)    

#### Step 2: import parameters from pytorch implement

In [551]:
# params = model.state_dict() 

# #remove # to see the params index
# for k,v in params.items():
#     #print parameter name
#     print(k,params[k].numpy().shape)

# conv1_weight = params['features.0.weight'].numpy().reshape(-1)
# conv1_bias = params['features.0.bias'].numpy()

#fire - fire - maxpool block 1
fire1_squeeze_weight = weights1.reshape(-1)
print(fire1_squeeze_weight)
fire1_squeeze_bias = bias1
print(fire1_squeeze_bias)
fire1_expand1x1_weight = weights2a.reshape(-1)
fire1_expand1x1_bias = bias2a

fire1_expand3x3_weight =weights2b.reshape(-1)
fire1_expand3x3_bias = bias2b

# fire2_squeeze_weight = params['features.4.squeeze.weight'].numpy().reshape(-1)
# fire2_squeeze_bias = params['features.4.squeeze.bias'].numpy()

# fire2_expand1x1_weight = params['features.4.expand1x1.weight'].numpy().reshape(-1)
# fire2_expand1x1_bias = params['features.4.expand1x1.bias'].numpy()

# fire2_expand3x3_weight = params['features.4.expand3x3.weight'].numpy().reshape(-1)
# fire2_expand3x3_bias = params['features.4.expand3x3.bias'].numpy()

# #fire - fire - maxpool block 2
# fire3_squeeze_weight = params['features.6.squeeze.weight'].numpy().reshape(-1)
# fire3_squeeze_bias = params['features.6.squeeze.bias'].numpy()

# fire3_expand1x1_weight = params['features.6.expand1x1.weight'].numpy().reshape(-1)
# fire3_expand1x1_bias = params['features.6.expand1x1.bias'].numpy()

# fire3_expand3x3_weight = params['features.6.expand3x3.weight'].numpy().reshape(-1)
# fire3_expand3x3_bias = params['features.6.expand3x3.bias'].numpy()

# fire4_squeeze_weight = params['features.7.squeeze.weight'].numpy().reshape(-1)
# fire4_squeeze_bias = params['features.7.squeeze.bias'].numpy()

# fire4_expand1x1_weight = params['features.7.expand1x1.weight'].numpy().reshape(-1)
# fire4_expand1x1_bias = params['features.7.expand1x1.bias'].numpy()

# fire4_expand3x3_weight = params['features.7.expand3x3.weight'].numpy().reshape(-1)
# fire4_expand3x3_bias = params['features.7.expand3x3.bias'].numpy()

# #fire - fire - fire - fire block 3
# fire5_squeeze_weight = params['features.9.squeeze.weight'].numpy().reshape(-1)
# fire5_squeeze_bias = params['features.9.squeeze.bias'].numpy()

# fire5_expand1x1_weight = params['features.9.expand1x1.weight'].numpy().reshape(-1)
# fire5_expand1x1_bias = params['features.9.expand1x1.bias'].numpy()

# fire5_expand3x3_weight = params['features.9.expand3x3.weight'].numpy().reshape(-1)
# fire5_expand3x3_bias = params['features.9.expand3x3.bias'].numpy()

# fire6_squeeze_weight = params['features.10.squeeze.weight'].numpy().reshape(-1)
# fire6_squeeze_bias = params['features.10.squeeze.bias'].numpy()

# fire6_expand1x1_weight = params['features.10.expand1x1.weight'].numpy().reshape(-1)
# fire6_expand1x1_bias = params['features.10.expand1x1.bias'].numpy()

# fire6_expand3x3_weight = params['features.10.expand3x3.weight'].numpy().reshape(-1)
# fire6_expand3x3_bias = params['features.10.expand3x3.bias'].numpy()

# fire7_squeeze_weight = params['features.11.squeeze.weight'].numpy().reshape(-1)
# fire7_squeeze_bias = params['features.11.squeeze.bias'].numpy()

# fire7_expand1x1_weight = params['features.11.expand1x1.weight'].numpy().reshape(-1)
# fire7_expand1x1_bias = params['features.11.expand1x1.bias'].numpy()

# fire7_expand3x3_weight = params['features.11.expand3x3.weight'].numpy().reshape(-1)
# fire7_expand3x3_bias = params['features.11.expand3x3.bias'].numpy()

# fire8_squeeze_weight = params['features.12.squeeze.weight'].numpy().reshape(-1)
# fire8_squeeze_bias = params['features.12.squeeze.bias'].numpy()

# fire8_expand1x1_weight = params['features.12.expand1x1.weight'].numpy().reshape(-1)
# fire8_expand1x1_bias = params['features.12.expand1x1.bias'].numpy()

# fire8_expand3x3_weight = params['features.12.expand3x3.weight'].numpy().reshape(-1)
# fire8_expand3x3_bias = params['features.12.expand3x3.bias'].numpy()

# classifier_conv_weight = params['classifier.1.weight'].numpy().reshape(-1)
# classifier_conv_bias = params['classifier.1.bias'].numpy()

[8. 6. 2. ... 4. 3. 1.]
[3. 1. 0. 3. 4. 3. 3. 6. 3. 1. 0. 0. 1. 7. 8. 2.]


Creat OpenCL memory object

In [552]:
h_sample = imagen.reshape(-1)
print(h_sample)
print(h_sample.shape)

[8 6 7 ... 7 1 9]
(387200,)


In [553]:
h_sample = imagen.reshape(-1).astype(np.float32)
# h_result_conv = np.empty(1 * 64 * 111 * 111).astype(np.float32)
# h_result_pool1 = np.empty(1 * 64 * 55 * 55).astype(np.float32)

h_result_fire1_squeeze = np.empty(1 * canales_contraidos * tamanyo * tamanyo).astype(np.float32)
h_result_fire1_expand = np.empty(1 * 2*canales_finales * tamanyo* tamanyo).astype(np.float32)

# h_result_fire2_squeeze = np.empty(1 * 16 * 55 * 55).astype(np.float32)
# h_result_fire2_expand = np.empty(1 * 128 * 55 * 55).astype(np.float32)
# h_result_pool2 = np.empty(1 * 128 * 27 * 27).astype(np.float32)

# h_result_fire3_squeeze = np.empty(1 * 32 * 27 * 27).astype(np.float32)
# h_result_fire3_expand = np.empty(1 * 256 * 27 * 27).astype(np.float32)
# h_result_fire4_squeeze = np.empty(1 * 32 * 27 * 27).astype(np.float32)
# h_result_fire4_expand = np.empty(1 * 256 * 27 * 27).astype(np.float32)
# h_result_pool3 = np.empty(1 * 256 * 13 * 13).astype(np.float32)

# h_result_fire5_squeeze = np.empty(1 * 48 * 13 * 13).astype(np.float32)
# h_result_fire5_expand = np.empty(1 * 384 * 13 * 13).astype(np.float32)
# h_result_fire6_squeeze = np.empty(1 * 48 * 13 * 13).astype(np.float32)
# h_result_fire6_expand = np.empty(1 * 384 * 13 * 13).astype(np.float32)
# h_result_fire7_squeeze = np.empty(1 * 64 * 13 * 13).astype(np.float32)
# h_result_fire7_expand = np.empty(1 * 512 * 13 * 13).astype(np.float32)
# h_result_fire8_squeeze = np.empty(1 * 64 * 13 * 13).astype(np.float32)
# h_result_fire8_expand = np.empty(1 * 512 * 13 * 13).astype(np.float32)

# h_result_classifier_conv = np.empty(1 * 1000 * 13 * 13).astype(np.float32)
# h_result_classifier = np.empty(1 * 1000).astype(np.float32)

# device input buffer
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

# device conv1 buffers 
# d_conv1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=conv1_weight)
# d_conv1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=conv1_bias)

# d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=weights1)
# d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=bias1)
# d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=weights2a)
# d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=bias2a)
# d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=weights2b)
# d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=bias2b)

print(d_fire1_squeeze_weight)
d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)
d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_weight)
d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_bias)
d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

# d_fire2_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire2_squeeze_weight)
# d_fire2_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire2_squeeze_bias)
# d_fire2_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire2_expand1x1_weight)
# d_fire2_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire2_expand1x1_bias)
# d_fire2_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire2_expand3x3_weight)
# d_fire2_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire2_expand3x3_bias)

# d_fire3_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire3_squeeze_weight)
# d_fire3_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire3_squeeze_bias)
# d_fire3_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire3_expand1x1_weight)
# d_fire3_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire3_expand1x1_bias)
# d_fire3_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire3_expand3x3_weight)
# d_fire3_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire3_expand3x3_bias)

# d_fire4_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire4_squeeze_weight)
# d_fire4_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire4_squeeze_bias)
# d_fire4_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire4_expand1x1_weight)
# d_fire4_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire4_expand1x1_bias)
# d_fire4_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire4_expand3x3_weight)
# d_fire4_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire4_expand3x3_bias)

# d_fire5_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire5_squeeze_weight)
# d_fire5_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire5_squeeze_bias)
# d_fire5_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire5_expand1x1_weight)
# d_fire5_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire5_expand1x1_bias)
# d_fire5_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire5_expand3x3_weight)
# d_fire5_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire5_expand3x3_bias)

# d_fire6_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire6_squeeze_weight)
# d_fire6_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire6_squeeze_bias)
# d_fire6_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire6_expand1x1_weight)
# d_fire6_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire6_expand1x1_bias)
# d_fire6_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire6_expand3x3_weight)
# d_fire6_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire6_expand3x3_bias)

# d_fire7_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire7_squeeze_weight)
# d_fire7_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire7_squeeze_bias)
# d_fire7_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire7_expand1x1_weight)
# d_fire7_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire7_expand1x1_bias)
# d_fire7_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire7_expand3x3_weight)
# d_fire7_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire7_expand3x3_bias)

# d_fire8_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire8_squeeze_weight)
# d_fire8_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire8_squeeze_bias)
# d_fire8_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire8_expand1x1_weight)
# d_fire8_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire8_expand1x1_bias)
# d_fire8_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire8_expand3x3_weight)
# d_fire8_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire8_expand3x3_bias)

# d_classifier_conv_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=classifier_conv_weight)
# d_classifier_conv_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=classifier_conv_bias)

# d_result_conv = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_conv.nbytes)
# d_result_pool1 = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_pool1.nbytes)

d_result_fire1_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_squeeze.nbytes)
d_result_fire1_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire1_expand.nbytes)
# d_result_fire2_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire2_squeeze.nbytes)
# d_result_fire2_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire2_expand.nbytes)
# d_result_pool2 = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_pool2.nbytes)

# d_result_fire3_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire3_squeeze.nbytes)
# d_result_fire3_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire3_expand.nbytes)
# d_result_fire4_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire4_squeeze.nbytes)
# d_result_fire4_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire4_expand.nbytes)
# d_result_pool3 = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_pool3.nbytes)

# d_result_fire5_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire5_squeeze.nbytes)
# d_result_fire5_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire5_expand.nbytes)
# d_result_fire6_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire6_squeeze.nbytes)
# d_result_fire6_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire6_expand.nbytes)
# d_result_fire7_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire7_squeeze.nbytes)
# d_result_fire7_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire7_expand.nbytes)
# d_result_fire8_squeeze = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire8_squeeze.nbytes)
# d_result_fire8_expand = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_fire8_expand.nbytes)

# d_result_classifier_conv = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_classifier_conv.nbytes)
# d_result_classifier = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_result_classifier.nbytes)

#### Step 3: creat kernels
Creat & build program

In [554]:
kernelSource = open("squeezenet.cl").read()
program = cl.Program(context, kernelSource).build()

Creat kernels

In [555]:
conv3x3 = program.conv2d3x3
conv3x3.set_scalar_arg_dtypes([np.int32, np.int32, np.int32, np.int32, np.int32, np.int32, \
                               None, None, None, None])

maxpool = program.maxpool2d
maxpool.set_scalar_arg_dtypes([np.int32, np.int32, None, None])

conv1x1 = program.conv2d1x1
conv1x1.set_scalar_arg_dtypes([np.int32, np.int32, \
                               None, None, None, None])

avgpool = program.avgpool2d
avgpool.set_scalar_arg_dtypes([None, None])

#### OpenCL kernel: squeezenet.cl
conv2d3x3: 2-D 3x3 convolution.  
conv2d1x1: 2-D 1x1 convolution. kerner size 1, stride 1  
maxpool2d: 2-D max pool. kerner size 3, stride 2  
avgpool2d: 2-D average pool. kerner size 13
```C
//maxPool2d 
//kernel_size=3 stride=2
//output one feature map per kernel
__kernel void maxpool2d(
	const int input_size,
	const int output_size,
	__global float *input_im,
    __global float *restrict output_im)
{
	int channel = get_global_id(0);//get output channel index
	
	input_im += channel * input_size * input_size;
	output_im += channel * output_size * output_size;

	//loop over output feature map
	for(int i = 0; i < output_size; i++)//row
	{
		for(int j = 0; j < output_size; j++)//col
		{
			//find the max value in 3x3 reigon 
			//to be one element in the output feature map
			float tmp = 0.0;

			for(int k = 0; k < 3; k++)//row
			{
				for(int l = 0; l < 3; l++)//col
				{
					float value = input_im[(i * 2 + k) * input_size  + j * 2 + l ];
					if(value > tmp)
						tmp = value;
				}
			}
			//store the result to output feature map
			output_im[i * output_size + j] = tmp; 
		}
	}
}

//3x3 convolution layer
//output one feature map per kernel
__kernel void conv2d3x3(
	const int input_channels, const int input_size,
	const int pad, const int stride,
	const int start_channel, //start_channel is for 1x1 feature map in fire layer
	const int output_size,
	__global float *restrict input_im,
	__global const float *restrict  filter_weight,
	__global const float *restrict  filter_bias,
	__global float *restrict output_im
	)
{
	int filter_index = get_global_id(0); //get output channel index

	filter_weight += filter_index * input_channels * 9;
	float bias = filter_bias[filter_index];
	output_im += (start_channel + filter_index) * output_size * output_size;
	
	//loop over output feature map
	for(int i = 0; i < output_size; i++)
	{
		for(int j = 0; j < output_size; j++)
		{
			//compute one element in the output feature map
			float tmp = bias;
			
			//compute dot product of 2 input_channels x 3 x 3 matrix
			for(int k = 0; k < input_channels; k++)
			{
				for(int l = 0; l < 3; l++)
				{
					int h = i * stride + l - pad;
					for(int m = 0; m < 3; m++)
					{
						int w = j * stride + m - pad;
						if((h >= 0) && (h < input_size) && (w >= 0) && (w < input_size))
						{
							tmp += input_im[k * input_size * input_size + (i * stride + l - pad) * input_size + j 
                                   * stride + m - pad] * filter_weight[9 * k + 3 * l + m];
						}
					}
				}
			}

			//add relu activation after conv
			output_im[i * output_size + j] = (tmp > 0.0) ? tmp : 0.0;
		}
	}
}

//1x1 convolution layer
//output one feature map per kernel
__kernel void conv2d1x1(
	const int input_channels, const int input_size,
	__global float *input_im,
	__global const float *restrict filter_weight,
	__global const float *restrict filter_bias,
	__global float *restrict output_im)
{
	int filter_index = get_global_id(0); // 0 - (output_channels - 1)

	filter_weight += filter_index * input_channels;

	float bias = filter_bias[filter_index];
	
	output_im += filter_index * input_size * input_size;//start_channel is for 1x1 feature map in fire layer

	//loop over output feature map
	//out
	for(int i = 0; i < input_size; i++)
	{
		for(int j = 0; j < input_size; j++)
		{
			float tmp = bias;
			for(int k = 0; k < input_channels; k++)
			{
				tmp += input_im[k * input_size * input_size + i * input_size + j] * filter_weight[k];
			}
			//add relu after conv
			output_im[i * input_size + j] = (tmp > 0.0) ? tmp : 0.0;
		}
	}
}

//last layer use a 13 x 13 avgPool layer as classifier
//one class score per kernel
__kernel void avgpool2d(
	__global float *restrict input_im,
	__global float *restrict output_im)
{
	int class_index = get_global_id(0);//get class score index

	input_im += 169 * class_index;
	
	float tmp = 0.0f;

	for(int i = 0; i < 169; i++)
	{
		tmp += input_im[i];
	}

	output_im[class_index] = tmp / 169.0;
}
```

Run OpenCL implement  

In [556]:
imagen = np.random.randint(10,size=(1,canales_iniciales, tamanyo, tamanyo))

weights1=np.random.randint(10,size=(canales_contraidos, canales_iniciales,1,1)).astype(np.float32)
bias1=np.random.randint(10,size=(canales_contraidos,)).astype(np.float32)
weights2a=np.random.randint(10,size=(canales_finales, canales_contraidos,1,1)).astype(np.float32)  
bias2a=np.random.randint(10,size=(canales_finales,)).astype(np.float32)    
weights2b=np.random.randint(10,size=(canales_finales, canales_contraidos,3,3)).astype(np.float32)    
bias2b=np.random.randint(10,size=(canales_finales,)).astype(np.float32)  

tic=pc()
squeeze1=nn.Conv2d(canales_iniciales, canales_contraidos, kernel_size=1, bias=False)
squeeze1.weight = nn.Parameter(torch.from_numpy(weights1))
squeeze1.bias = nn.Parameter(torch.from_numpy(bias1))

squeeze2a=nn.Conv2d(canales_contraidos, canales_finales, kernel_size=1, bias=False)
squeeze2a.weight = nn.Parameter(torch.from_numpy(weights2a))
squeeze2a.bias = nn.Parameter(torch.from_numpy(bias2a))

squeeze2b=nn.Conv2d(canales_contraidos, canales_finales, kernel_size=3, bias=False, padding=1)
squeeze2b.weight = nn.Parameter(torch.from_numpy(weights2b))
squeeze2b.bias = nn.Parameter(torch.from_numpy(bias2b))


imagen1  = torch.from_numpy(imagen).float()

salida1=squeeze1(imagen1)
salida1_activation=squeeze_activation(salida1)

salida2a=squeeze2a(salida1_activation)
salida2a_activation=squeeze_activation(salida2a)
salida2b=squeeze2b(salida1_activation)
salida2b_activation=squeeze_activation(salida2b)    
salida2_total=torch.cat([salida2a_activation,salida2b_activation], 1)
salida2_total_a_numpy=salida2_total.detach().numpy()
toc=pc()
acumulado_pytorch=toc-tic+acumulado_pytorch


tic2=pc()
n_h=1
n_w=1
pesos1=np.transpose(weights1, axes=(1, 0, 2, 3)).reshape(canales_iniciales*n_h*n_w, -1)
pesos2a=np.transpose(weights2a, axes=(1, 0, 2, 3)).reshape(canales_contraidos*n_h*n_w, -1)

n_h=3
n_w=3
idea2b_pre=weights2b.reshape(-1,canales_contraidos*n_h*n_w)
pesos2b=np.transpose(idea2b_pre)   
bt, c_i = imagen.shape[0:2] 
s=1
n_h=1
n_w=1
k=imagen.shape[2]-n_h+1
col = np.zeros((bt, c_i , k,k, n_h, n_w))

for i in range(k):
    i_max = i + s*n_h
    for j in range(k):
        j_max = j + s*n_w
        col[:, :, i, j, :, :] = imagen[:, :, i:i_max:s, j:j_max:s]
# col = np.transpose(col, axes=(1, 4, 5, 0, 2, 3)).reshape(c_i*n_h*n_w, -1)
col = col.reshape(c_i*n_h*n_w, -1)
imagen1_numpy=np.transpose(col)
salida1_obtenida= np.dot(imagen1_numpy,pesos1)+bias1
salida1_obtenida_activacion=np.maximum(0,salida1_obtenida)
imagen1_obtenida = np.transpose(salida1_obtenida_activacion, axes=(1,0)).reshape(canales_contraidos,tamanyo,tamanyo)
imagen1_obtenida_ampliada=np.pad(imagen1_obtenida,((0,0),(1,1),(1,1)))
salida2a_obtenida= np.dot(salida1_obtenida,pesos2a)+bias2a
salida2a_obtenida_activacion=np.maximum(0,salida2a_obtenida)
c_i = imagen1_obtenida_ampliada.shape[0] 
n_h=3
n_w=3
k=imagen1_obtenida_ampliada.shape[1]-n_h+1
col = np.zeros(( c_i , k,k, n_h, n_w))

for i in range(k):
    i_max = i + s*n_h
    for j in range(k):
        j_max = j + s*n_w
        col[ :, i, j, :, :] = imagen1_obtenida_ampliada[ :, i:i_max:s, j:j_max:s]
col = np.transpose(col, axes=(0, 3, 4, 1, 2)).reshape(c_i*n_h*n_w, -1)
#col = col.reshape(c_i*n_h*n_w, -1)
salida_obtenida_2b=np.transpose(col)
salida2b_obtenida= np.dot(salida_obtenida_2b,pesos2b)+bias2b
salida2b_obtenida_activacion=np.maximum(0,salida2b_obtenida)
salida2_total_numpy=np.concatenate((salida2a_obtenida_activacion,salida2b_obtenida_activacion), axis=1)
imagen_total_obtenida = np.transpose(salida2_total_numpy, axes=(1,0)).reshape(canales_finales*2,tamanyo,tamanyo)
toc2=pc()
acumulado_numpy=toc2-tic2+acumulado_numpy
idea&=np.allclose(salida2_total_a_numpy, imagen_total_obtenida,rtol=1e-01, atol=1e-01)


# device conv1 buffers 
# d_conv1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=conv1_weight)
# d_conv1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=conv1_bias)

# d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=weights1)
# d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=bias1)
# d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=weights2a)
# d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=bias2a)
# d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=weights2b)
# d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=bias2b)

h_sample = imagen.reshape(-1).astype(np.float32)
d_sample = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_sample)

fire1_squeeze_weight = weights1.reshape(-1)
fire1_squeeze_bias = bias1
fire1_expand1x1_weight = weights2a.reshape(-1)
fire1_expand1x1_bias = bias2a
fire1_expand3x3_weight =weights2b.reshape(-1)
fire1_expand3x3_bias = bias2b

d_fire1_squeeze_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_weight)
d_fire1_squeeze_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_squeeze_bias)
d_fire1_expand1x1_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_weight)
d_fire1_expand1x1_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand1x1_bias)
d_fire1_expand3x3_weight = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_weight)
d_fire1_expand3x3_bias = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=fire1_expand3x3_bias)

tic3 = pc()
#first conv layer
# conv3x3(queue,(64,), None, 3, 224, 0, 2, 0, 111, d_sample, d_conv1_weight, d_conv1_bias, d_result_conv)
# maxpool(queue,(64,), None, 111, 55, d_result_conv, d_result_pool1)

#block1
conv1x1(queue,(canales_contraidos,), None, canales_iniciales, tamanyo, d_sample, d_fire1_squeeze_weight, d_fire1_squeeze_bias, d_result_fire1_squeeze)
conv1x1(queue,(canales_finales,), None, canales_contraidos, tamanyo, d_result_fire1_squeeze, d_fire1_expand1x1_weight, d_fire1_expand1x1_bias, d_result_fire1_expand)
conv3x3(queue,(canales_finales,), None, canales_contraidos, tamanyo, 1, 1, canales_finales, tamanyo, d_result_fire1_squeeze, d_fire1_expand3x3_weight, d_fire1_expand3x3_bias, d_result_fire1_expand)

# conv1x1(queue,(16,), None, 128, 55, d_result_fire1_expand, d_fire2_squeeze_weight, d_fire2_squeeze_bias, d_result_fire2_squeeze)
# conv1x1(queue,(64,), None, 16, 55, d_result_fire2_squeeze, d_fire2_expand1x1_weight, d_fire2_expand1x1_bias, d_result_fire2_expand)
# conv3x3(queue,(64,), None, 16, 55, 1, 1, 64, 55, d_result_fire2_squeeze, d_fire2_expand3x3_weight, d_fire2_expand3x3_bias, d_result_fire2_expand)

# maxpool(queue,(128,), None, 55, 27, d_result_fire2_expand, d_result_pool2)

# #block2
# conv1x1(queue,(32,), None, 128, 27, d_result_pool2, d_fire3_squeeze_weight, d_fire3_squeeze_bias, d_result_fire3_squeeze)
# conv1x1(queue,(128,), None, 32, 27, d_result_fire3_squeeze, d_fire3_expand1x1_weight, d_fire3_expand1x1_bias, d_result_fire3_expand)
# conv3x3(queue,(128,), None, 32, 27, 1, 1, 128, 27, d_result_fire3_squeeze, d_fire3_expand3x3_weight, d_fire3_expand3x3_bias, d_result_fire3_expand)

# conv1x1(queue,(32,), None, 256, 27, d_result_fire3_expand, d_fire4_squeeze_weight, d_fire4_squeeze_bias, d_result_fire4_squeeze)
# conv1x1(queue,(128,), None, 32, 27, d_result_fire4_squeeze, d_fire4_expand1x1_weight, d_fire4_expand1x1_bias, d_result_fire4_expand)
# conv3x3(queue,(128,), None, 32, 27, 1, 1, 128, 27, d_result_fire4_squeeze, d_fire4_expand3x3_weight, d_fire4_expand3x3_bias, d_result_fire4_expand)

# maxpool(queue,(256,), None, 27, 13, d_result_fire4_expand, d_result_pool3)

# #block3
# conv1x1(queue,(48,), None, 256, 13, d_result_pool3, d_fire5_squeeze_weight, d_fire5_squeeze_bias, d_result_fire5_squeeze)
# conv1x1(queue,(192,), None, 48, 13, d_result_fire5_squeeze, d_fire5_expand1x1_weight, d_fire5_expand1x1_bias, d_result_fire5_expand)
# conv3x3(queue,(192,), None, 48, 13, 1, 1, 192, 13, d_result_fire5_squeeze, d_fire5_expand3x3_weight, d_fire5_expand3x3_bias, d_result_fire5_expand)

# conv1x1(queue,(48,), None, 384, 13, d_result_fire5_expand, d_fire6_squeeze_weight, d_fire6_squeeze_bias, d_result_fire6_squeeze)
# conv1x1(queue,(192,), None, 48, 13, d_result_fire6_squeeze, d_fire6_expand1x1_weight, d_fire6_expand1x1_bias, d_result_fire6_expand)
# conv3x3(queue,(192,), None, 48, 13, 1, 1, 192, 13, d_result_fire6_squeeze, d_fire6_expand3x3_weight, d_fire6_expand3x3_bias, d_result_fire6_expand)

# conv1x1(queue,(64,), None, 384, 13, d_result_fire6_expand, d_fire7_squeeze_weight, d_fire7_squeeze_bias, d_result_fire7_squeeze)
# conv1x1(queue,(256,), None, 64, 13, d_result_fire7_squeeze, d_fire7_expand1x1_weight, d_fire7_expand1x1_bias, d_result_fire7_expand)
# conv3x3(queue,(256,), None, 64, 13, 1, 1, 256, 13, d_result_fire7_squeeze, d_fire7_expand3x3_weight, d_fire7_expand3x3_bias, d_result_fire7_expand)

# conv1x1(queue,(64,), None, 512, 13, d_result_fire7_expand, d_fire8_squeeze_weight, d_fire8_squeeze_bias, d_result_fire8_squeeze)
# conv1x1(queue,(256,), None, 64, 13, d_result_fire8_squeeze, d_fire8_expand1x1_weight, d_fire8_expand1x1_bias, d_result_fire8_expand)
# conv3x3(queue,(256,), None, 64, 13, 1, 1, 256, 13, d_result_fire8_squeeze, d_fire8_expand3x3_weight, d_fire8_expand3x3_bias, d_result_fire8_expand)

# # classifier
# conv1x1(queue,(1000,), None, 512, 13, d_result_fire8_expand, d_classifier_conv_weight, d_classifier_conv_bias, d_result_classifier_conv)
# avgpool(queue,(1000,), None, d_result_classifier_conv, d_result_classifier)
# # Wait for the commands to finish before reading back
# event3.wait()
queue.finish()
#cl.enqueue_copy(queue, h_result_fire1_squeeze,  d_result_fire1_squeeze)
cl.enqueue_copy(queue, h_result_fire1_expand, d_result_fire1_expand)
queue.finish()
veamos=h_result_fire1_expand.reshape(-1,tamanyo,tamanyo)
#veamos2=h_result_fire1_squeeze.reshape(-1,tamanyo,tamanyo)
rtime = pc() - tic3
print ("tiempo en segundos con pytorch= ", toc-tic)
print ("tiempo en segundos con numpy= ", toc2-tic2)
print ("tiempo en segundos con opencl=",rtime)

comparativa1=np.allclose(salida2_total_a_numpy, imagen_total_obtenida,rtol=1e-01, atol=1e-01)
comparativa2=np.allclose(veamos, imagen_total_obtenida,rtol=1e-01, atol=1e-01)
print("comparativa (pytorch==numpy):",comparativa1)
print("comparativa (pytorch==opencl):",comparativa2)

tiempo en segundos con pytorch=  0.01650763675570488
tiempo en segundos con numpy=  0.09821171686053276
tiempo en segundos con opencl= 0.10182949528098106
comparativa (pytorch==numpy): True
comparativa (pytorch==opencl): True


In [557]:
print(imagen_total_obtenida.shape)
print(veamos.shape)
# print(imagen)
# print(weights1)
# print(fire1_squeeze_weight)
# print(bias1)
# print(imagen1_obtenida)
# print(veamos2)

(128, 55, 55)
(128, 55, 55)
